### model brand

In [1]:
import pandas as pd

In [6]:
def load_df_products(name):
    return pd.read_csv(f'../data/processed/data_beer_{name}.csv')

In [7]:
data_beer_carrefour = load_df_products('carrefour')
data_beer_alcampo = load_df_products('alcampo')
data_beer_corteingles = load_df_products('corteingles')
data_beer_dia = load_df_products('dia')
data_beer_eroski = load_df_products('eroski')

In [20]:
aa = [data_beer_carrefour, data_beer_alcampo]

In [19]:
data_beer_eroski['brand'].unique()

array(['SAN MIGUEL', 'MAHOU C', 'MAHOU 5 E', ' MAHOU', ' 1925 ALHAMBRA',
       'ALHAMBRA T', 'CORONITA', ' BUDWEISER', ' CORONA', '0 SAN MIGUEL',
       ' ALHAMBRA', ' FRANZISKANER', 'MAHOU M', ' MAHOU 5 E',
       'SAN MIGUEL S', ' LEFFE', ' LA SALVE', 'ALHAMBRA E',
       ' GRIMBERGEN DOUBLE', 'MAHOU S', ' SAN MIGUEL', ' LA SALVE B',
       'KRONENBOURG', 'CARLSBERG', ' GRIMBERGEN BLONDE', ' LA SALVE M',
       ' 1925 ALHAMBRA ', ' FRANZISKANER D', 'BUDWEISER', ' FOUNDERS',
       'SAN MIGUEL 1516', 'ABADÍA LEFFE B', 'WARSTEINER', ' SAN MIGUEL ',
       ' GRIMBERGEN', ' MODELO', 'PACÍFICO', 'APA T', ' SPATEM MUNC',
       ' BECK', 'CUBANISTO', ' ESTRELLA GALICIA', 'AURUM', 'VOLL',
       'HEINEKEN', 'AMSTEL R', '1906', 'AMSTEL', 'KELER',
       '1906 ESTRELLA GALICIA', ' AMSTEL R', '0 ESTRELLA GALICIA',
       ' HEINEKEN', ' PAULANER H', 'AMSTEL O', 'ESTRELLA DAMM', 'WOLL',
       ' AURUM', 'AMBAR', ' CRUZCAMPO S', 'CRUZCAMPO S', '0 R', '1906 R',
       ' DESPERADOS', '0 AMSTEL O', 